Find table size from linear probing load factor with average of 1.5 comparisions for an unsuccessful search of a hash table with 100 objects:

1.5 = 1/2[1 + 1/(1 - λ)^2]
λ = 0.3

Load factor (λ) = (number of objects) / (number of buckets)
Table size = number of buckets = 100 / λ = 333.33
next prime = 337

Find table size from double hashing load factor with average of 1.5 comparisions for an unsuccessful search of a hash table with 100 objects:

1.5 = 1/(1 - λ)
λ = 1/3

Load factor (λ) = (number of objects) / (number of buckets)
Table size = number of buckets = 100 / λ = 300
next prime = 307

In [20]:
import random
import math

class TableEntry:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.in_table = True

    def get_key(self):
        return self.key

    def get_value(self):
        return self.value

    def set_value(self, value):
        self.value = value

    def is_in_table(self):
        return self.in_table

    def set_to_removed(self):
        self.key = None
        self.value = None
        self.in_table = False

    def is_removed(self):
        return (self.key is None) and (self.value is None) and not self.in_table


In [21]:
class HashTableWithCount(): # abstract data type

    def __init__(self, initialCapacity):
        self.table = [None] * initialCapacity # make a hash table full of empty objects based on capacity
        self.table_size = initialCapacity
        self.count = 0
        self.probe_count = 0

    def is_prime(self, n):
        if n <= 1:
            return False
        for i in range(2, int(n**0.5) + 1): # loop from 2 to the square root of n, if n % i == 0, it's not prime, if no divisors found, n is prime
            if n % i == 0:
                return False
        return True

    def get_next_prime(self, n):
        while not self.is_prime(n):
            n += 1
        return n
    
    def check_size(self, size):
        if self.table_size < size:
            return False
        return True
    
    def get_hash_index(self, key):
        return hash(key) % self.table_size # hash the key to get the index
    
    def is_hash_table_too_full(self):
        return self.count / self.table_size > 0.5

    def enlarge_hash_table(self):
        temp_table = self.table
        next_size = self.get_next_prime(self.table_size * 2)
        self.table = [None] * next_size
        self.table_size = next_size
        self.count = 0
        for entry in temp_table:
            if entry is not None and entry.in_table:
                self.add(entry.key, entry. value)
    
    def reset_probe_count(self):
        self.probe_count = 0

    def get_probes(self):
        return self.probe_count


In [22]:
class LinearProbingWithCount(HashTableWithCount):

    def probe(self, index, key):
        return (index + 1) % self.table_size # hashing equation index = (index + 1) % self.table_size
    
    def add(self, key, value):
        index = self.get_hash_index(key)
        while True: # infinite loop
            self.probe_count += 1 # increase probing count
            entry = self.table[index] # pick our slot
            if entry is None or entry.is_removed(): # check if slot is empty or has been emptied
                self.table[index] = TableEntry(key, value)
                self.count += 1
                return True # break loop
            if entry.in_table and entry.key == key: # check if slot contains a real entry and if the key matches
                entry.set_value(value)
                return True # break loop
            index = self.probe(index, key) # continue linear probing

    def remove(self, key):
        index = self.get_hash_index(key)
        while True:
            self.probe_count += 1
            entry = self.table[index]
            if entry is None:
                return False
            if entry.in_table and entry.key == key:
                entry.set_to_removed()
                self.count -= 1
                return True
            index = self.probe(index, key)

    def get_value(self, key):
        index = self.get_hash_index(key)
        while True:
            self.probe_count += 1
            entry = self.table[index]
            if entry is None:
                return None
            if entry.in_table and entry.key == key:
                return entry.value
            index = self.probe(index, key)

    def contains(self, key):
        return self.get_value(key) is not None # runs get value and checks if slot is empty

    def is_empty(self):
        return self.count == 0

    def get_size(self):
        return self.count

    def clear(self):
        self.table = [None] * self.table_size # empties every slot
        self.count = 0
        self.reset_probe_count()


In [23]:
class DoubleHashingWithCount(HashTableWithCount):
    
    def primary_hash(self, key): # determine our slot
        """First hash function."""
        return hash(key) % self.table_size
    
    def secondary_hash(self, key): # determine our step size
        """Second hash function to calculate step size."""
        return 1 + (hash(key) % (self.table_size - 1))

    def probe(self, index, key): # when probing we must use our secondary hash for collisions
        return (index + self.secondary_hash(key)) % self.table_size

    def add(self, key, value):
        index = self.primary_hash(key)
        while True:
            self.probe_count += 1
            entry = self.table[index]
            if entry is None or entry.is_removed():
                self.table[index] = TableEntry(key, value)
                self.count += 1
                return True
            if entry.in_table and entry.key == key:
                entry.set_value(value)
                return True
            index = self.probe(index, key)

    def remove(self, key):
        index = self.primary_hash(key)
        while True:
            self.probe_count += 1
            entry = self.table[index]
            if entry is None:
                return False
            if entry.in_table and entry.key == key:
                entry.set_to_removed()
                self.count -= 1
                return True
            index = self.probe(index, key)

    def get_value(self, key):
        index = self.primary_hash(key)
        while True:
            self.probe_count += 1
            entry = self.table[index]
            if entry is None:
                return None
            if entry.in_table and entry.key == key:
                return entry.value
            index = self.probe(index, key)

    def contains(self, key):
        return self.get_value(key) is not None

    def is_empty(self):
        return self.count == 0

    def get_size(self):
        return self.count

    def clear(self):
        self.table = [None] * self.table_size
        self.count = 0
        self.reset_probe_count()


In [29]:
import random
import statistics

class GetStatistics:

    def main():
        linear_ht = LinearProbingWithCount(337)
        double_ht = DoubleHashingWithCount(307)
        linear_counts = []
        double_counts = []
        names_1000 = GetStatistics.get_1000_names([])
        names_10000 = GetStatistics.get_10000_names([])
        for _ in range(1000):
            linear_ht.clear() # clear the hash tables
            double_ht.clear()
            GetStatistics.choose_100_names_and_add(linear_ht, double_ht, names_1000) # randomly choose 100 names from the list of 1000 and insert them into the tables
            linear_count, double_count = GetStatistics.search_for_100_names(linear_ht, double_ht, names_10000) # randomly choose 100 names from the list of 10,000 and search the tables for each name
            linear_counts.append(linear_count) # count the number of comparisions in each table for the 100 searches
            double_counts.append(double_count)
        linear_avg = sum(linear_counts) / 1000 # average comparisions = compairisons / searches
        print(f"Linear Probing Average Comparisions: {linear_avg:.3f}")
        linear_standard_deviation = statistics.stdev(linear_counts) # find standard deviation from list of comparisions
        print(f"Std Dev: {linear_standard_deviation:.3f}")
        double_avg = sum(double_counts) / 1000
        print(f"Average comparisons (Double Hashing): {double_avg:.3f}")
        double_standard_deviation = statistics.stdev(double_counts)
        print(f"Std Dev: {double_standard_deviation:.3f}")



    def choose_100_names_and_add(linear_ht, double_ht, names):
        chosen = random.sample(names, 100) # randomly chooses 100 names from names list
        for name in chosen:
            linear_ht.add(name, 1)
            double_ht.add(name, 1)

    def get_1000_names(names):
        return [f"name_{i}" for i in range(1000)] # creates 1000 name_0, name_1, ...

    def get_10000_names(names):
        return [f"othername_{i}" for i in range(10000)] # we want unsuccesful searches so comparing othername_x vs name_x, creates 10000 othername_0, othername_1, ...
    
    def search_for_100_names(linear_ht, double_ht, names):
        chosen = random.sample(names, 100) # randomly chooses 100 names from othernames list
        linear_ht.reset_probe_count()
        double_ht.reset_probe_count()
        for name in chosen:
            linear_ht.contains(name)
            double_ht.contains(name)
        return linear_ht.get_probes(), double_ht.get_probes()

if __name__ == "__main__":
    GetStatistics.main()


Linear Probing Average Comparisions: 150.726
Std Dev: 11.334
Average comparisons (Double Hashing): 148.341
Std Dev: 8.384


From this data we can see that double hashing leads to fewer collisions in a hash table. This is due to the fact that linear probing will search each index sequentially for an empty slot. This collision handling introduces clustering as data will always be slot to slot increasing the chance of future collisions. Double hashing instead spreads data in a hash table by handling collisions by determining the next position for the data with another hashing function.